### Import libralies

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import joblib
import pickle

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Conv1D, MaxPooling1D, Reshape
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

### Functions for data preprocesing

In [ ]:
# load dataset
def load_dataset():
    dataset = read_data('actitracker_raw.txt')
    dataset.dropna(axis=0, how='any', inplace= True)
    dataset.drop_duplicates(['user_id','activity','timestamp', 'x-axis', 'y-axis', 'z-axis'], keep= 'first', inplace= True)
    dataset['x-axis'] = feature_normalize(dataset['x-axis'])
    dataset['y-axis'] = feature_normalize(dataset['y-axis'])
    dataset['z-axis'] = pd.to_numeric(dataset['z-axis'].str.replace(';',''))
    dataset['z-axis'] = feature_normalize(dataset['z-axis'])
    return dataset    

# read data
def read_data(file_path):
    column_names = ['user_id','activity','timestamp', 'x-axis', 'y-axis', 'z-axis']
    data = pd.read_csv(file_path,header = None, names = column_names)
    return data

# normalize data
def feature_normalize(dataset):
    mu = np.mean(dataset,axis = 0)
    sigma = np.std(dataset,axis = 0)
    return (dataset - mu)/sigma

# slide windows
def windows(data, size):
    start = 0
    while start < data.count():
        yield int(start), int(start + size)
        start += (size / 2)

# segment signal
def segment_signal(data,window_size = 128):
    segments = np.empty((0,window_size,3))
    labels = np.empty((0))
    for (start, end) in windows(data['timestamp'], window_size):
        x = data["x-axis"][start:end]
        y = data["y-axis"][start:end]
        z = data["z-axis"][start:end]
        if(len(dataset['timestamp'][start:end]) == window_size):
            segments = np.vstack([segments,np.dstack([x,y,z])])
            labels = np.append(labels,stats.mode(data["activity"][start:end])[0][0])
    return segments, labels

### Functions for evaluations

In [ ]:
# build the model and evaluation
def evaluate_model(train_x, train_y, test_x, test_y):

    verbose, epochs, batch_size = 0, 15, 32
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
    model = Sequential()

    learning_rate = 0.001 # With the default value of learning rate
    l2_rate = 1e-4
    input_shape = Input(shape=(n_timesteps,n_features))
    pool_size = 2
    
    x = Conv1D(32, kernel_size = 24, 
            activation = "relu", 
            padding = "valid", 
            kernel_regularizer = l2(l2_rate))(input_shape)
    x = Dropout(0.5)(x)
    x = MaxPooling1D(pool_size)(x)
    
    x = Conv1D(64, kernel_size = 16, 
            activation = "relu", 
            padding = "valid", 
            kernel_regularizer = l2(l2_rate))(x)
    x = Dropout(0.5)(x)
    x = MaxPooling1D(pool_size)(x)
    
    x = Conv1D(96, kernel_size = 8, 
            activation = "relu", 
            padding = "valid", 
            kernel_regularizer = l2(l2_rate))(x)
    x = Dropout(0.5)(x)
    x = MaxPooling1D(pool_size)(x)
    
    x = Conv1D(128, kernel_size = 4, 
            activation = "relu", 
            padding = "valid", 
            kernel_regularizer = l2(l2_rate),
            name="encoder")(x)
    x = Flatten()(x) 
    output = Dense(n_outputs, activation = "softmax")(x)
    
    model = Model(input_shape, output)
    model.compile(loss='categorical_crossentropy', optimizer="adam", 
                  metrics=[AUC()])
    
    # simple early stopping
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
    
    # fit network
    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose, 
              validation_split=0.33, callbacks=[es])
    
    # evaluate model
    _, accuracy = model.evaluate(test_x, test_y, batch_size=batch_size, verbose=0)
    
    # return results
    return model, accuracy

# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = np.mean(scores), np.std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

# run an experiment
def run_experiment(repeats, train_x, train_y, test_x, test_y):
    
    # repeat experiment
    scores = list()
    for r in range(repeats):
        model, score = evaluate_model(train_x, train_y, test_x, test_y)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    
    # summarize results
    summarize_results(scores)
    return model, scores

### Run experiments

In [ ]:
dataset = read_data('data/dataset.txt')
users = [17,35, 20, 6, 15, 28, 5, 10, 18, 11, 34, 1, 24, 26, 12, 32, 31, 13]
window_size = 128
n_features = 3
users_all = []
scores_all = []

# train the CNN by each user
for user in users:
    print("------------------------", user, "------------------------")
    
    dataset = load_dataset()
    dataset = dataset[dataset["user_id"] == user]
    print(dataset.shape)
    
    segments, labels = segment_signal(dataset)
    labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)
    reshaped_segments = segments.reshape(len(segments), window_size , n_features)

    # shuffle and split training and test sets
    random_state = 42
    train_test_split = np.random.rand(len(reshaped_segments)) < 0.70
    train_x = reshaped_segments[train_test_split]
    train_y = labels[train_test_split]
    test_x = reshaped_segments[~train_test_split]
    test_y = labels[~train_test_split]

    model, scores = run_experiment(repeats=10, train_x=train_x, train_y=train_y, test_x=test_x, test_y=test_y)
    
    for score in scores:
        users_all.append(user)
        scores_all.append(score)

    print("----------------------------------------------------------")
    

### Export results to CSV

In [ ]:
# export accuracies
data = {'user_id':users_all,
        'accuracy':scores_all,
       }
df = pd.DataFrame(data=data)
df.to_csv('accuracies.csv', index=False)

In [ ]:
# export methods
method = {'uid':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10,  1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        'user_id':[users],
        'method':["Proposed", "Proposed", "Proposed", "Proposed", "Proposed", "Proposed", "Proposed", "Proposed", "Proposed", "Proposed",
                 "Baseline", "Baseline", "Baseline", "Baseline", "Baseline", "Baseline", "Baseline", "Baseline", "Baseline", "Baseline"],
       }
df_method = pd.DataFrame(data=method)
df_method.to_csv('method.csv', index=False)